In [2]:
import torch
print(torch.__version__)

import onnxscript
print(onnxscript.__version__)

from onnxscript import opset18  # opset 18 is the latest (and only) supported version for now

import onnxruntime
print(onnxruntime.__version__)

2.5.1+cu124
0.1.0.dev20241216
1.19.2


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
from model import PersonalityNet

IMAGE_SIZE = 224
MODEL_PATH = 'personality_net_v0.pth'
torch_model = PersonalityNet(40)
torch_model.load_state_dict(torch.load(MODEL_PATH))
torch_model.eval()
torch_input = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/home/balezz/PycharmProjects/VKR_2026/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/balezz/PycharmProjects/VKR_2026/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_73572/2750361320.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pyt

In [19]:
onnx_program.save("personality_net_v0.onnx")

In [20]:
import onnx
onnx_model = onnx.load("personality_net_v0.onnx")
onnx.checker.check_model(onnx_model)

In [21]:
import onnxruntime

onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("personality_net_v0.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)


Input length: 1
Sample input: (tensor([[[[ 0.6128,  0.4241, -0.5568,  ...,  0.0998,  0.3581,  1.0123],
          [ 0.4136, -0.3911, -0.6002,  ..., -1.9885, -1.4222, -1.9069],
          [-1.1729, -1.0350,  0.5237,  ...,  0.8811, -1.8240, -0.8810],
          ...,
          [ 0.8228, -0.5605,  0.3120,  ...,  1.1548, -0.2013,  0.0983],
          [ 1.3331,  1.5856,  0.9334,  ...,  0.3839,  0.0333, -0.5055],
          [ 0.7549,  1.5181, -0.0996,  ..., -0.6737, -1.6479,  0.1103]],

         [[-0.1834, -2.2012, -0.3332,  ...,  0.1364, -0.1190, -1.8318],
          [ 1.0314,  0.5691,  0.3286,  ..., -0.6144, -0.5421, -0.0503],
          [ 1.0776, -0.0555, -0.8719,  ...,  0.2611,  1.3041, -2.7974],
          ...,
          [-0.2891,  0.9216, -0.2106,  ...,  0.6908, -0.4912,  0.0441],
          [ 2.0351, -1.7855,  1.2020,  ..., -0.9580,  0.3642,  1.0284],
          [-0.1556,  0.7434, -1.1849,  ...,  0.7105, -0.5426, -0.8888]],

         [[ 1.6284,  0.1900,  0.3663,  ..., -0.6372,  1.0656,  0.4692],

In [22]:
print(onnxruntime_outputs)

[array([[7.05268979e-03, 9.72339511e-03, 5.19418716e-03, 6.47196174e-03,
        4.34219837e-05, 1.13058090e-03, 1.30260646e-01, 1.46382809e-01,
        1.07228935e-01, 1.54344141e-02, 9.08637524e-01, 3.37916613e-03,
        3.08096409e-04, 8.10871899e-01, 8.22763443e-01, 6.32267892e-01,
        2.96709239e-02, 2.85762548e-03, 3.27401459e-01, 1.95557475e-02,
        1.79436117e-01, 7.60744214e-02, 1.38558447e-02, 1.48195326e-02,
        5.08105695e-01, 5.14079034e-02, 7.07417727e-04, 1.25536919e-01,
        1.52488053e-02, 5.96624613e-03, 1.96450055e-02, 7.05426335e-02,
        8.11780274e-01, 8.47219229e-02, 4.00946110e-01, 2.69618630e-02,
        3.46796751e-01, 5.01515687e-01, 8.49515200e-04, 2.90677607e-01]],
      dtype=float32)]
